In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc, precision_score, recall_score, precision_recall_curve, log_loss
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import label_binarize


In [ ]:


# Load the Fashion MNIST dataset
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()


# Filter the data to use only classes from 0 to 4
mask_train = np.isin(train_labels, [0, 1, 2, 3, 4])
train_images_filtered = train_images[mask_train]
train_labels_filtered = train_labels[mask_train]

mask_test = np.isin(test_labels, [0, 1, 2, 3, 4])
test_images_filtered = test_images[mask_test]
test_labels_filtered = test_labels[mask_test]


In [ ]:


# Reshape the images into vectors
train_images_filtered = train_images_filtered.reshape((train_images_filtered.shape[0], 28 * 28))
test_images_filtered = test_images_filtered.reshape((test_images_filtered.shape[0], 28 * 28))


# Normalize image data
train_images_filtered, test_images_filtered = train_images_filtered / 255.0, test_images_filtered / 255.0


# Initialize and train the Logistic Regression model
model = LogisticRegression(max_iter=1000)
model.fit(train_images_filtered, train_labels_filtered)


In [ ]:


# Make predictions on the test data
predictions = model.predict(test_images_filtered)


# Calculate the log-loss (will act as the "Loss Curve")
train_probs = model.predict_proba(train_images_filtered)  # Probabilities for the training set
test_probs = model.predict_proba(test_images_filtered)    # Probabilities for the test set

train_log_loss = log_loss(train_labels_filtered, train_probs)
test_log_loss = log_loss(test_labels_filtered, test_probs)

print(f"Train Log-Loss: {train_log_loss:.4f}")
print(f"Test Log-Loss: {test_log_loss:.4f}")


# Plot the Log-Loss curve with enhanced aesthetics
plt.figure(figsize=(8, 6))


# Plot the Log-Loss curve for training and test sets
plt.plot([train_log_loss, test_log_loss], label="Log-Loss", marker='o', color='b', linestyle='-', markersize=10, markerfacecolor='red')


# Add labels for the axes
plt.xlabel('Epochs', fontsize=14)
plt.ylabel('Log-Loss', fontsize=14)


# Add a title to the plot
plt.title('Log-Loss Curve for Training and Testing', fontsize=16)


# Add grid lines for easier reading
plt.grid(True, which='both', linestyle='--', linewidth=0.5)


# Add annotations or labels for the points with Log-Loss values clearly
plt.text(0, train_log_loss, f'{train_log_loss:.4f}', horizontalalignment='right', verticalalignment='bottom', fontsize=12, color='blue')
plt.text(1, test_log_loss, f'{test_log_loss:.4f}', horizontalalignment='left', verticalalignment='top', fontsize=12, color='blue')


# Add a Log-Loss box on the left side of the plot
plt.text(-0.2, (train_log_loss + test_log_loss) / 2, f"Train Log-Loss: {train_log_loss:.4f}\nTest Log-Loss: {test_log_loss:.4f}",
         horizontalalignment='right', verticalalignment='center', fontsize=12, color='white',
         bbox=dict(facecolor='darkblue', edgecolor='blue', boxstyle='round,pad=1'))


# Add a legend to the plot
plt.legend(loc="upper right", fontsize=12)


# Show the plot
plt.tight_layout()
plt.show()


In [ ]:


# Calculate the confusion matrix
cm = confusion_matrix(test_labels_filtered, predictions)
print("Confusion Matrix:")
print(cm)


# Classification report
print("Classification Report:")
print(classification_report(test_labels_filtered, predictions))


In [ ]:


# Other metrics
accuracy = np.mean(predictions == test_labels_filtered)
precision = precision_score(test_labels_filtered, predictions, average='weighted')
recall = recall_score(test_labels_filtered, predictions, average='weighted')

print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")


In [ ]:


# Convert the labels to a binary format for ROC curve plotting
y_test_bin = label_binarize(test_labels_filtered, classes=np.arange(5))


# Calculate the ROC curve
fpr = {}
tpr = {}
roc_auc = {}

for i in range(5):
    fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], model.predict_proba(test_images_filtered)[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])


# Plot the ROC curves
plt.figure(figsize=(10, 8))
colors = ['aqua', 'darkorange', 'green', 'red', 'blue']

for i, color in zip(range(5), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=2, label=f'Class {i} (AUC = {roc_auc[i]:0.2f})')

plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves for Multiple Classes')
plt.legend(loc="lower right")
plt.tight_layout()
plt.show()


# Display performance metrics
plt.figure(figsize=(6, 4))
plt.text(0.5, 0.8, f'Accuracy: {accuracy * 100:.2f}%', ha='center', fontsize=16, color='green', bbox=dict(facecolor='lightgreen', edgecolor='green', boxstyle='round,pad=1'))
plt.text(0.5, 0.6, f'Precision: {precision:.2f}', ha='center', fontsize=16, color='blue', bbox=dict(facecolor='lightblue', edgecolor='blue', boxstyle='round,pad=1'))
plt.text(0.5, 0.4, f'Recall: {recall:.2f}', ha='center', fontsize=16, color='red', bbox=dict(facecolor='lightcoral', edgecolor='red', boxstyle='round,pad=1'))

plt.axis('off')
plt.title('Model Performance Metrics', fontsize=18)
plt.show()


In [ ]:


# Plot Precision-Recall curves
plt.figure(figsize=(10, 8))
for i, color in zip(range(5), colors):
    precision_vals, recall_vals, _ = precision_recall_curve(y_test_bin[:, i], model.predict_proba(test_images_filtered)[:, i])
    plt.plot(recall_vals, precision_vals, color=color, lw=2, label=f'PR curve for class {i} (Precision-Recall)')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curves for Multiple Classes')
plt.legend(loc="lower left")
plt.show()


In [ ]:


# Plot the confusion matrix
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=np.arange(5), yticklabels=np.arange(5))
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix for 5 Classes')
plt.show()



In [ ]:

# Display some example images
plt.figure(figsize=(10, 10))
for i in range(9):
    plt.subplot(3, 3, i + 1)
    plt.imshow(test_images_filtered[i].reshape(28, 28), cmap='gray')
    plt.title(f"True: {test_labels_filtered[i]}, Pred: {predictions[i]}")
    plt.axis('off')
plt.tight_layout()
plt.show()